In [1]:
!pip install --upgrade certifi
!pip install census us pandas geopandas tqdm requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.9/356.9 kB 7.8 MB/s eta 0:00:00


In [2]:
import requests, zipfile, io, os, tempfile, shutil, geopandas as gpd
from census import Census
from us import states
import pandas as pd
import certifi, os
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib import animation, colormaps, colors
from pathlib import Path
import shutil
from google.colab import files

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
# ---------- 1. PULL ALL ACS-5 YEARS ----------
API_KEY      = "8546100ded13cfc85c2c2bf7d6695324b321d280"          # ← put your key here
c            = Census(API_KEY)
state_fips   = states.TX.fips                 # "48"
county_fips  = "303"                          # Lubbock
years        = range(2009, 2024)              # 2009-2023 ACS-5
vars         = ["B02001_001E","B02001_002E","B02001_003E","B02001_004E",
                "B02001_005E","B02001_006E","B02001_007E","B02001_008E"]

records = []
for yr in tqdm(years, desc="ACS 5-yr fetch"):
    data = c.acs5.state_county_tract(vars, state_fips, county_fips, tract="*", year=yr)
    df   = pd.DataFrame(data)
    df   = df[df["tract"] != "980000"]        # drop pseudo-tract row
    df["year"] = yr
    records.append(df)

df = pd.concat(records, ignore_index=True)
df = df.rename(columns={
    "B02001_001E":"Total","B02001_002E":"White","B02001_003E":"Black",
    "B02001_004E":"Native_American","B02001_005E":"Asian",
    "B02001_006E":"Pacific_Islander","B02001_007E":"Other",
    "B02001_008E":"Two_or_more"
})
df["GEOID"] = df["state"] + df["county"] + df["tract"]

df.to_csv("lubbock_race_acs5_2009_2023.csv", index=False)
print("✓ Race CSV saved.")



ACS 5-yr fetch: 100%|██████████| 15/15 [00:35<00:00,  2.39s/it]

✓ Race CSV saved.


In [4]:
# ---------- 2. DOWNLOAD & FILTER SHAPEFILE ----------
TX_TRACT_URL = "https://www2.census.gov/geo/tiger/TIGER2023/TRACT/tl_2023_48_tract.zip"
tmp_dir = tempfile.mkdtemp()

print("Downloading Texas tracts shapefile …")
r = requests.get(TX_TRACT_URL, verify=False)
zipfile.ZipFile(io.BytesIO(r.content)).extractall(tmp_dir)

shp_path = [f for f in os.listdir(tmp_dir) if f.endswith(".shp")][0]
gdf_tx   = gpd.read_file(os.path.join(tmp_dir, shp_path))

gdf_lb   = gdf_tx[gdf_tx["COUNTYFP"] == county_fips].copy()
print(f"✓ Filtered to {len(gdf_lb)} Lubbock tracts.")

# save shapefile & GeoJSON
out_dir = "lubbock_tracts"
os.makedirs(out_dir, exist_ok=True)
gdf_lb.to_file(os.path.join(out_dir, "lubbock_tracts_2023.shp"))
gdf_lb.to_file(os.path.join(out_dir, "lubbock_tracts_2023.geojson"), driver="GeoJSON")

# ---------- 3. MERGE 2023 COUNTS WITH GEOMETRY ----------
race_2023 = df[df["year"] == 2023].copy()
gdf_lb["GEOID"] = gdf_lb["GEOID"].astype(str)
gdf_merge      = gdf_lb.merge(race_2023, on="GEOID", how="left")

# save a GeoPackage (handy single file)
gdf_merge.to_file("lubbock_race_map_2023.gpkg", layer="race2023", driver="GPKG")
print("✓ Shapefile, GeoJSON, and merged GeoPackage saved.")

# ---------- 4. CLEANUP ----------
shutil.rmtree(tmp_dir)
print("All done!")


/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www2.census.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


✓ Filtered to 106 Lubbock tracts.
✓ Shapefile, GeoJSON, and merged GeoPackage saved.
All done!


In [5]:
# ------------------------------------------------------------------
# 0)  Ensure GEOID keys are strings
# ------------------------------------------------------------------
gdf_lb["GEOID"] = gdf_lb["GEOID"].astype(str)
df["GEOID"]     = df["GEOID"].astype(str)

years = sorted(df["year"].unique())          # [2009 … 2023]

# ------------------------------------------------------------------
# 1)  Define the race columns & nice colormaps
# ------------------------------------------------------------------
race_info = {
    "White"            : ("White",            "Blues"),
    "Black"            : ("Black",            "OrRd"),
    "Native_American"  : ("Native_American",  "YlGn"),
    "Asian"            : ("Asian",            "Purples"),
    "Pacific_Islander" : ("Pacific_Islander", "Greens"),
    "Other"            : ("Other",            "Greys"),
    "Two_or_more"      : ("Two_or_more",      "BuPu"),
}

# ------------------------------------------------------------------
# 2)  Build one GIF per race
# ------------------------------------------------------------------
for race_label, (col_name, cmap_name) in race_info.items():

    # ---- build GeoDataFrames list (one per year) ----
    gdfs, vmax = [], 0
    for yr in years:
        g = gdf_lb.merge(
                df[df["year"] == yr][["GEOID", col_name]],
                on="GEOID", how="left")
        g[col_name] = pd.to_numeric(g[col_name], errors="coerce")  # NaN ok
        vmax = max(vmax, g[col_name].max(skipna=True))
        gdfs.append(g)

    # ---- set up colormap & figure ----
    cmap = colormaps.get_cmap(cmap_name).copy()
    cmap.set_bad("white")
    norm = colors.Normalize(vmin=0, vmax=vmax)

    fig, ax = plt.subplots(figsize=(6.5, 6.5))
    ax.set_axis_off()

    # add a fixed color bar on the right
    cax = fig.add_axes([0.87, 0.25, 0.03, 0.5])  # [left, bottom, width, height]
    sm  = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
    sm.set_array([])
    cb  = fig.colorbar(sm, cax=cax)
    cb.set_label(f"{race_label} population", fontsize=9)
    cb.ax.tick_params(labelsize=8)

    # ---- animation update function ----
    def update(frame):
        ax.clear()
        ax.set_axis_off()
        gdfs[frame].plot(
            column=col_name, cmap=cmap, norm=norm,
            edgecolor="black", linewidth=0.35, ax=ax
        )
        ax.set_title(f"Lubbock County – {race_label}\nACS 5-Year {years[frame]}",
                     fontweight="bold", fontsize=11)

    ani = animation.FuncAnimation(
        fig, update, frames=len(years),
        interval=1000, blit=False, repeat=True
    )

    out_gif = f"lubbock_{col_name.lower()}_pop_2009_2023.gif"
    ani.save(out_gif, writer="pillow", dpi=150)
    plt.close(fig)
    print(f"✓ Saved {out_gif}")


✓ Saved lubbock_white_pop_2009_2023.gif
✓ Saved lubbock_black_pop_2009_2023.gif
✓ Saved lubbock_native_american_pop_2009_2023.gif
✓ Saved lubbock_asian_pop_2009_2023.gif
✓ Saved lubbock_pacific_islander_pop_2009_2023.gif
✓ Saved lubbock_other_pop_2009_2023.gif
✓ Saved lubbock_two_or_more_pop_2009_2023.gif


In [ ]:
# ---------- 0) prep keys ----------
gdf_lb["GEOID"] = gdf_lb["GEOID"].astype(str)
df["GEOID"]     = df["GEOID"].astype(str)
years           = sorted(df["year"].unique())

race_info = {
    "White"            : ("White",            "Blues"),
    "Black"            : ("Black",            "OrRd"),
    "Native_American"  : ("Native_American",  "YlGn"),
    "Asian"            : ("Asian",            "Purples"),
    "Pacific_Islander" : ("Pacific_Islander", "Greens"),
    "Other"            : ("Other",            "Greys"),
    "Two_or_more"      : ("Two_or_more",      "BuPu"),
}

root_dir = Path("lubbock_race_frames")
root_dir.mkdir(exist_ok=True)

# ---------- 1) loop races ----------
for race_label, (col_name, cmap_name) in race_info.items():

    race_dir = root_dir / race_label
    race_dir.mkdir(exist_ok=True)

    # compute common vmax across all years
    vmax = df[col_name].astype(float).max()
    cmap = colormaps.get_cmap(cmap_name).copy()
    cmap.set_bad("white")
    norm = colors.Normalize(vmin=0, vmax=vmax)

    for yr in years:
        g = gdf_lb.merge(
            df[df["year"] == yr][["GEOID", col_name]],
            on="GEOID", how="left"
        )
        g[col_name] = pd.to_numeric(g[col_name], errors="coerce")

        fig, ax = plt.subplots(figsize=(6.5, 6.5))
        ax.set_axis_off()
        g.plot(column=col_name, cmap=cmap, norm=norm,
               edgecolor="black", linewidth=0.35, ax=ax)

        # colorbar
        cax = fig.add_axes([0.87, 0.25, 0.03, 0.5])
        sm  = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
        sm.set_array([])
        cb  = fig.colorbar(sm, cax=cax)
        cb.set_label(f"{race_label} population", fontsize=8)
        cb.ax.tick_params(labelsize=7)

        ax.set_title(f"Lubbock County – {race_label}\nACS 5-Year {yr}",
                     fontweight='bold', fontsize=11)
        plt.tight_layout()

        out_path = race_dir / f"{race_label}_{yr}.png"
        plt.savefig(out_path, dpi=150, bbox_inches="tight")
        plt.close(fig)

    print(f"✓ Saved {race_label} frames → {race_dir}")

# ---------- 2) zip the folder ----------
shutil.make_archive("lubbock_race_frames", "zip", root_dir)
print("✓ Zipped as lubbock_race_frames.zip")

# ---------- 3) download ----------
files.download("lubbock_race_frames.zip")


/tmp/ipython-input-21-3060759787.py:53: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/tmp/ipython-input-21-3060759787.py:53: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/tmp/ipython-input-21-3060759787.py:53: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/tmp/ipython-input-21-3060759787.py:53: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/tmp/ipython-input-21-3060759787.py:53: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/tmp/ipython-input-21-3060759787.py:53: UserWarning: This figure includes Axes that are not compatible with tight_layout, so resul

✓ Saved White frames → lubbock_race_frames/White


/tmp/ipython-input-21-3060759787.py:53: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/tmp/ipython-input-21-3060759787.py:53: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/tmp/ipython-input-21-3060759787.py:53: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/tmp/ipython-input-21-3060759787.py:53: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/tmp/ipython-input-21-3060759787.py:53: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/tmp/ipython-input-21-3060759787.py:53: UserWarning: This figure includes Axes that are not compatible with tight_layout, so resul

✓ Saved Black frames → lubbock_race_frames/Black


/tmp/ipython-input-21-3060759787.py:53: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/tmp/ipython-input-21-3060759787.py:53: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/tmp/ipython-input-21-3060759787.py:53: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/tmp/ipython-input-21-3060759787.py:53: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/tmp/ipython-input-21-3060759787.py:53: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/tmp/ipython-input-21-3060759787.py:53: UserWarning: This figure includes Axes that are not compatible with tight_layout, so resul

...

✓ Saved Other frames → lubbock_race_frames/Other


/tmp/ipython-input-21-3060759787.py:53: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/tmp/ipython-input-21-3060759787.py:53: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/tmp/ipython-input-21-3060759787.py:53: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/tmp/ipython-input-21-3060759787.py:53: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/tmp/ipython-input-21-3060759787.py:53: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/tmp/ipython-input-21-3060759787.py:53: UserWarning: This figure includes Axes that are not compatible with tight_layout, so resul

✓ Saved Two_or_more frames → lubbock_race_frames/Two_or_more
✓ Zipped as lubbock_race_frames.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
import folium

# Ensure the GeoDataFrame is in WGS84 (lat/lon) for web mapping
gdf_display = gdf_lb.to_crs(epsg=4326)

# Center the map on the centroid of all Lubbock tracts
centroid = gdf_display.geometry.unary_union.centroid
m = folium.Map(location=[centroid.y, centroid.x], zoom_start=11, tiles="cartodbpositron")

# Add tract polygons with a tooltip that shows only the GEOID
folium.GeoJson(
    gdf_display,
    style_function=lambda feature: {
        "fillColor": "#3186cc",
        "color": "#3186cc",
        "weight": 1,
        "fillOpacity": 0.2,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["GEOID"],
        aliases=["Tract GEOID:"],
        sticky=True,
    ),
    name="Lubbock Census Tracts",
).add_to(m)

# Optional: add layer control if you plan to add more layers
folium.LayerControl().add_to(m)

# Display the interactive map (in a notebook this will render inline)
m


/tmp/ipython-input-1888516896.py:7: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  centroid = gdf_display.geometry.unary_union.centroid


In [9]:
m.save('my_folium_map.html')